In [1]:
%pip install -U langgraph langchain_community langchain_anthropic langchain_experimental

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install python-dotenv langchain_groq langchain_ollama


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:

import getpass
import os
from dotenv import find_dotenv, load_dotenv
from langchain_groq import ChatGroq
from langchain_ollama import ChatOllama

dotenv_path = find_dotenv()
load_dotenv(dotenv_path)
groq_api_key = os.getenv("groq_api_key")
os.environ["groq_api_key"] = groq_api_key 

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("groq_api_key")

dotenv_path = find_dotenv()
load_dotenv(dotenv_path)
tavily_api_key = os.getenv("tavily_api_key")
os.environ["tavily_api_key"] = tavily_api_key

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("tavily_api_key")

In [4]:
from typing import Annotated

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
from langchain_experimental.utilities import PythonREPL

tavily_tool = TavilySearchResults(max_results=5)

# This executes code locally, which can be unsafe
repl = PythonREPL()


@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """Use this to execute python code and do math. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    result_str = f"Successfully executed:\n\`\`\`python\n{code}\n\`\`\`\nStdout: {result}"
    return result_str

<>:23: SyntaxWarning: invalid escape sequence '\`'
<>:23: SyntaxWarning: invalid escape sequence '\`'
<>:23: SyntaxWarning: invalid escape sequence '\`'
<>:23: SyntaxWarning: invalid escape sequence '\`'
C:\Users\paulo\AppData\Local\Temp\ipykernel_4132\2391770398.py:23: SyntaxWarning: invalid escape sequence '\`'
  result_str = f"Successfully executed:\n\`\`\`python\n{code}\n\`\`\`\nStdout: {result}"
C:\Users\paulo\AppData\Local\Temp\ipykernel_4132\2391770398.py:23: SyntaxWarning: invalid escape sequence '\`'
  result_str = f"Successfully executed:\n\`\`\`python\n{code}\n\`\`\`\nStdout: {result}"


In [8]:
from typing import Literal
from typing_extensions import TypedDict

from langchain_anthropic import ChatAnthropic
from langgraph.graph import MessagesState, END
from langgraph.types import Command


members = ["researcher", "coder", "expenses_manager"]
# Our team supervisor is an LLM node. It just picks the next agent to process
# and decides when the work is completed
options = members + ["FINISH"]

system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    f" following workers: {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)


class Router(TypedDict):
    """Worker to route to next. If no workers needed, route to FINISH."""

    next: Literal[*options]


#llm = ChatAnthropic(model="claude-3-5-sonnet-latest")

#llm=ChatGroq(groq_api_key=groq_api_key,model_name="Llama-3.3-70b-Versatile")

MODEL_NAME = "deepseek-r1:latest"
#MODEL_NAME = "llama3.2:latest"
llm = ChatOllama(model=MODEL_NAME, temperature=0.5)

MODEL_NAME = "llama3.2:latest"
llm_tool = ChatOllama(model=MODEL_NAME, temperature=0.5)
class State(MessagesState):
    next: str

    
call_count = 0
call_count_max = 2
def supervisor_node(state: State) -> Command[Literal[*members, "__end__"]]:
    global call_count  # Now it refers to the global variable
    global call_count_max

    call_count = call_count +1

    messages = [
        {"role": "system", "content": system_prompt},
    ] + state["messages"]
    response = llm.with_structured_output(Router).invoke(messages)
    goto = response["next"]
    if goto == "FINISH":
        goto = END

    if call_count  == call_count_max:
        goto = END

    return Command(goto=goto, update={"next": goto})

In [9]:
result = llm.invoke("state")

In [10]:
result

AIMessage(content='<think>\n\n</think>\n\nThe term "state" can refer to various concepts depending on the context. Here are some common meanings:\n\n1. **Government Entity**: A state is a sovereign political entity that exercises power over an area, typically including territory, people, and government institutions.\n\n2. **Condition or Stage**: It can describe something in its natural condition (e.g., "the state of the environment") or as a stage of development (e.g., "the state of technology").\n\n3. **Legal System**: In legal contexts, "state" often refers to a sovereign state within the international system, governing its own affairs.\n\n4. **Computing**: In computing, "state" can refer to a condition or mode resulting from the execution of a program, which may depend on previous inputs or events.\n\n5. **Biology**: In biology, "state" can describe an organism\'s condition or health (e.g., "the state of the ecosystem").\n\n6. **General Usage**: It can also be used more generally to

In [ ]:
result

In [7]:
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import create_react_agent


research_agent = create_react_agent(
    llm_tool, tools=[tavily_tool], prompt="You are a researcher. DO NOT do any math."
)


def research_node(state: State) -> Command[Literal["supervisor"]]:
    #print("state:",state)
    result = research_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="researcher")
            ]
        },
        goto="supervisor",
    )


# NOTE: THIS PERFORMS ARBITRARY CODE EXECUTION, WHICH CAN BE UNSAFE WHEN NOT SANDBOXED
code_agent = create_react_agent(llm, tools=[python_repl_tool])


def code_node(state: State) -> Command[Literal["supervisor"]]:
    #print("state:",state)
    result = code_agent.invoke(state)

    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="coder")
            ]
        },
        goto="supervisor",
    )


import json
# Load the JSON file
file_path = "monthly_expenses.json"  # Replace with the correct path to your file
with open(file_path, "r") as file:
    data = json.load(file)

# Convert the JSON data to a string
monthly_expenses_json_string = json.dumps(data, indent=4)  # Pretty-printed string

expenses_manager_agent = create_react_agent(llm, tools=[],prompt=f"You are a expenses manager. you data is solely based here{monthly_expenses_json_string}")

def expenses_manager_node(state: State) -> Command[Literal["supervisor"]]:

    result = expenses_manager_agent.invoke(state)

    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="expenses_manager")
            ]
         
        },
        goto="supervisor",
    )


builder = StateGraph(State)
builder.add_edge(START, "supervisor")
builder.add_node("supervisor", supervisor_node)
builder.add_node("researcher", research_node)
builder.add_node("coder", code_node)
builder.add_node("expenses_manager", expenses_manager_node)
graph = builder.compile()

In [15]:
from IPython.display import display, Image

#display(Image(graph.get_graph().draw_mermaid_png()))

In [16]:
call_count = 0
call_count_max = 5
events = graph.stream(
    {"messages": [("user", "What's the square root of 42?")]}, subgraphs=True
)

In [17]:
my_array = []
for s in events:
    print(s)
    my_array.append(s)
    print("----")

((), {'supervisor': {'next': 'researcher'}})
----
state: {'messages': [HumanMessage(content="What's the square root of 42?", additional_kwargs={}, response_metadata={}, id='5c9577ff-7b4f-4313-ad00-802679db68a4')], 'next': 'researcher'}
(('researcher:2720ff28-132b-8c39-0a78-e5e8e3e005a6',), {'agent': {'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:latest', 'created_at': '2025-04-02T02:26:18.887474Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4526373000, 'load_duration': 3903171500, 'prompt_eval_count': 205, 'prompt_eval_duration': 229000000, 'eval_count': 25, 'eval_duration': 387000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-1e028666-480a-4cd2-96fe-e087cced3ffe-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'square root of 42'}, 'id': 'ace1d6f6-fbf1-4e13-9124-ab8ee6dab9dc', 'type': 'tool_call'}], usage_metadata={'input_tokens': 205, 'output_tokens': 25

In [21]:
agents = []
j = -2
for i in range(len(my_array)):
    
    try:
        if my_array[i][1]['supervisor']['next']:  # Check if 'next' key exists and is not empty
            

            if (j == -2):
                agent =  my_array[i][1]['supervisor']['next']
                print("AGENT:", agent)


            j = j+1
           
            agent =  my_array[i][1]['supervisor']['next']
         
            agents.append(agent)

            #agent = agent.append(my_array[i][1]['supervisor']['next'])
            print(my_array[i - 1][1][agents[j]]["messages"][0].content)
            print("-----------")

            print("AGENT:", agent)
            
            
            
    except (KeyError, IndexError, TypeError):
        pass


AGENT: researcher
The square root of 42 is approximately 6.481, which cannot be reduced further and is an irrational number.
-----------
AGENT: researcher
I apologize for the mistake. It seems like I don't have direct access to the math library. However, I can try searching for the square root of 42.

According to my search results, the square root of 42 is approximately 6.481, which cannot be reduced further and is an irrational number.
-----------
AGENT: researcher
The square root of 42 is approximately 6.481, which cannot be reduced further and is an irrational number.
-----------
AGENT: researcher
The square root of 42 is approximately 6.481, which cannot be reduced further and is an irrational number.
-----------
AGENT: __end__


In [24]:
call_count = 0
call_count_max = 5
events = graph.stream(
    {"messages": [("user", "What's the square root of 42?")]}, subgraphs=True
)

my_array = []
for s in events:
    
    my_array.append(s)

agents = []
j = -2
for i in range(len(my_array)):
    
    try:
        if my_array[i][1]['supervisor']['next']:  # Check if 'next' key exists and is not empty
            

            if (j == -2):
                agent =  my_array[i][1]['supervisor']['next']
                print("AGENT:", agent)


            j = j+1
           
            agent =  my_array[i][1]['supervisor']['next']
         
            agents.append(agent)

            #agent = agent.append(my_array[i][1]['supervisor']['next'])
            print(my_array[i - 1][1][agents[j]]["messages"][0].content)
            print("-----------")

            print("AGENT:", agent)
            
            
            
    except (KeyError, IndexError, TypeError):
        pass

AGENT: researcher
It seems like I need to use the `tavily` search results instead of the `math.sqrt` function. Let me try again.

Unfortunately, it looks like 42 is not a perfect square, and its square root cannot be expressed as a simple integer or fraction. If you'd like, I can provide an approximate decimal value for you!
-----------
AGENT: researcher
The square root of 42 is approximately 6.481.
-----------
AGENT: researcher
The square root of 42 is approximately 6.481.
-----------
AGENT: __end__


In [8]:
def multi_agent_bot(prompt):
    global call_count
    global call_count_max

    call_count = 0
    call_count_max = 5
    events = graph.stream(
        {"messages": [("user", prompt)]}, subgraphs=True
    )

    my_array = []
    for s in events:
        print(s)
        my_array.append(s)

    agents = []
    j = -2
    for i in range(len(my_array)):
        
        try:
            if my_array[i][1]['supervisor']['next']:  # Check if 'next' key exists and is not empty
                

                if (j == -2):
                    agent =  my_array[i][1]['supervisor']['next']
                    print("AGENT:", agent)


                j = j+1
            
                agent =  my_array[i][1]['supervisor']['next']
            
                agents.append(agent)

                #agent = agent.append(my_array[i][1]['supervisor']['next'])
                print(my_array[i - 1][1][agents[j]]["messages"][0].content)
                print("-----------")

                print("AGENT:", agent)
                
                
                
        except (KeyError, IndexError, TypeError):
            pass

In [9]:
prompt = "what is average age of dog search the internet then give me a summary of the expenses"
multi_agent_bot(prompt)

((), {'supervisor': {'next': 'researcher'}})
(('researcher:e2c19577-2967-89f5-48b9-de5806baa823',), {'agent': {'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:latest', 'created_at': '2025-04-02T03:04:22.6635571Z', 'done': True, 'done_reason': 'stop', 'total_duration': 473574400, 'load_duration': 16795300, 'prompt_eval_count': 213, 'prompt_eval_duration': 18000000, 'eval_count': 24, 'eval_duration': 433000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-61d421b7-548b-4474-933d-438cf091519a-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'average age of dog + internet + expenses'}, 'id': 'a31c0fce-7a91-4f31-a1e5-77039e45a4d7', 'type': 'tool_call'}], usage_metadata={'input_tokens': 213, 'output_tokens': 24, 'total_tokens': 237})]}})
(('researcher:e2c19577-2967-89f5-48b9-de5806baa823',), {'tools': {'messages': [ToolMessage(content='[{"title": "Here Are the True Costs of Pet 

Python REPL can execute arbitrary code. Use with caution.


(('coder:454c9505-9bab-3ac7-9245-db7cea4e03bd',), {'agent': {'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:latest', 'created_at': '2025-04-02T03:04:43.2337118Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2489053500, 'load_duration': 14292800, 'prompt_eval_count': 819, 'prompt_eval_duration': 295000000, 'eval_count': 127, 'eval_duration': 2175000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-cd6520e6-48dd-42c8-bf6e-c4e590fbb507-0', tool_calls=[{'name': 'python_repl_tool', 'args': {'code': "import statistics\naverage_age = 11.5\nprint(f'The average lifespan of dogs is approximately {average_age} years.')\nprint('The estimated annual expenses for a dog are:\\nd Monthly Essential Expenses: $127.75/month\\nd Annual Food Costs: $339/year\\nd Veterinary Care: $367/year\\nd Grooming: $99/year\\nd Dog Boarding (7-day vacation): $253/year\\nTotal estimated annual expenses: approximately $

In [10]:
prompt = "give me a summary of the expenses"
multi_agent_bot(prompt)

((), {'supervisor': {'next': 'researcher'}})
(('researcher:e18dc0ff-0415-d5c8-a76f-8f5b8ca5e216',), {'agent': {'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:latest', 'created_at': '2025-04-02T03:04:51.4964135Z', 'done': True, 'done_reason': 'stop', 'total_duration': 445039400, 'load_duration': 13747400, 'prompt_eval_count': 203, 'prompt_eval_duration': 15000000, 'eval_count': 23, 'eval_duration': 413000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-10b920e7-9326-47c3-9d1e-3676ec82a927-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'summary of expenses'}, 'id': '3d4432e6-669c-4213-9456-b0fd611a4e77', 'type': 'tool_call'}], usage_metadata={'input_tokens': 203, 'output_tokens': 23, 'total_tokens': 226})]}})
(('researcher:e18dc0ff-0415-d5c8-a76f-8f5b8ca5e216',), {'tools': {'messages': [ToolMessage(content='[{"title": "How to Create an Expenses Summary: A Step-by-Step Gu